In [27]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import datetime as dt

## twitter automatic login
참고: https://jhnoru.tistory.com/59

In [15]:
"""
driver 가동
: web browser는 chrome 사용
"""
def init_driver():
    driver = webdriver.Chrome()
    driver.wait = WebDriverWait(driver, 5)
    
    return driver


def close_driver(driver):
    
    return

In [16]:
"""
크롬을 조작해서 트위터에 로그인
: 개인 id 및 pw이용
"""
def twitter_login(driver, username, password):
    driver.get("https://twitter.com/login")
    driver.implicitly_wait(3)
    driver.maximize_window()
    
    username_field = driver.find_element_by_name("session[username_or_email]")
    password_field = driver.find_element_by_name("session[password]")

    username_field.send_keys(username)
    driver.implicitly_wait(1)
    password_field.send_keys(password)
    driver.implicitly_wait(1)

    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[3]/div/div/span/span').submit()


In [24]:
def crawling(driver):
    data = []
    TWITTER_URL = "https://twitter.com/search?q=%EA%B2%BD%EC%A0%9C%20until%3A2020-09-29%20since%3A2020-09-28%20-filter%3Alinks&src=typed_query"

    driver.maximize_window()
    driver.get(TWITTER_URL)

    print('The scroll is starting to move bottom')

    # 페이지 스크롤을 끝날 때까지 계속 내림
    # 스크롤을 내리기 전의 화면 높이와 내렸을 때의 화면 높이가 같다면 더 이상 내려갈 곳이 없다는 의미이므로 무한 루프를 탈출함.
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            # Wait to load page
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if(new_height == last_height):
                break

        last_height = new_height
    print('Arrived at the end of the page')
    print('Start twitter crawling')
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 트위터의 게시글에는 각각의 고유의 아이디가 있어서 트위터 사이트에서 모든 게시글 아이디를 리스트형으로 가져옴
    pattern = re.compile('stream-item-tweet-\d+')
    items = pattern.findall(str(soup))
    for item in items:
        # 위에서 가져온 고유 아이디를 이용하여 게시글 본문을 css selector를 이용하여 가져
        text = driver.find_element_by_css_selector('#'+ item +' > div > div.content > div.js-tweet-text-container > p').text

        # 특수기호를 없애는 작업
        for idx in range(len(text)):
            if not ((0 <= ord(text[idx]) < 128) or (0xac00 <= ord(text[idx]) <= 0xd7af)):
                text = text.replace(text[idx], ' ')

        data.append(text)

    
    print('Finish crawling')
    print('The data is being written to the csv file.')
    dataframe = pd.DataFrame(data, columns=["content"])
    dataframe.to_csv('twitter_comment.csv', mode = 'a', encoding='cp949')
    print('Finish working')

In [25]:
#driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver')
driver = init_driver()
twitter_login(driver, 'twitter_user_id', 'twitter_user_pw')

crawling(driver)

driver.quit()

The scroll is starting to move bottom
Arrived at the end of the page
Start twitter crawling
Finish crawling
The data is being written to the csv file.
Finish working


InvalidSessionIdException: Message: invalid session id


자동 로그인까지 성공.\
검색어 query로 크롤링 시도하고자함.

### 혜승 크롤링 연습

In [58]:
def crawling(driver, start_year, start_month, start_day, until_year, until_month, until_day):
    startdate = dt.date(year = start_year,month = start_month, day = start_day)
    untildate = dt.date(year = until_year,month = until_month, day = until_day)

    #TWITTER_URL = "https://twitter.com/search?q=%EA%B2%BD%EC%A0%9C%20until%3A2020-09-30%20since%3A2020-09-29%20-filter%3Alinks&src=typed_query"
    
    TWITTER_URL = "https://twitter.com/search?q=%EA%B2%BD%EC%A0%9C%20until%3A" + str(untildate) + "%20since%3A"+ str(startdate) +"%20-filter%3Alinks&src=typed_query"

    driver.maximize_window()
    driver.get(TWITTER_URL)

    print('The scroll is starting to move bottom')

    # 페이지 스크롤을 끝날 때까지 계속 내림
    # 스크롤을 내리기 전의 화면 높이와 내렸을 때의 화면 높이가 같다면 더 이상 내려갈 곳이 없다는 의미이므로 무한 루프를 탈출함.
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            # Wait to load page
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if(new_height == last_height):
                break

        last_height = new_height
    print('Arrived at the end of the page')
    print('Start twitter crawling')
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 날짜 저장된 태그: day
    day = soup.find_all("a", {"class": "r-1re7ezh r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0 css-4rbku5 css-18t94o4 css-901oao"})
    tweets = soup.find_all("div", {"class": "css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0"})
    return day, tweets

In [59]:
driver = init_driver()
twitter_login(driver, 'twitter_user_id', 'twitter_user_pw')

day, tweets = crawling(driver, 2020, 9, 29, 2020, 9, 30)
driver.quit()

The scroll is starting to move bottom
Arrived at the end of the page
Start twitter crawling


전체 글이 긁어와지지않음. 더 분석해봐야할듯! + 키워드 조정

In [60]:
# 날짜 저장된 태그
for a in day:
    print(a.get_text())

9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일
9월 30일


In [63]:
for a in tweets:
    print(a.get_text())
    print()
    print()

승뉸이 슈스케 끝나고 경제적으로 힘든 상태였는데 광고 찍은거랑 행사 뛴거 미리 정산해줘서 빚 갚았대.. 회사에 돈이 많으니 그런 쪽으로는 클린한 듯..


[효민] 향수는 달콤한 향이나 바닐라 향을 좋아한다. (한국경제-스타들의 수다)


카와세가 산리오를 좋아하는 것도... 어찌보면 당시의 상황적 맥락이 좀 작용하죠 일본 경제가 잃어버린 10년(요즘은 20년이라 합니다)을 겪는 동안 산리오의 매출도 위터러웠어요 캐릭터 상품이 고가에 팔리던 시절인데, 이때부터 한정된 캐릭터가 아닌 보다 다양한 캐릭터를, 여러 가격대에 맞춰


뭐 중요한 건 카와세네 아버지 집안과 어머니 집안은 사이가 좋지 않습니다.... 아버지 쪽은 거품 경제 당시 나름 건실했던 토목공사 사업들의 수익을 부동산에 투자했다 이후 망한 뒤 미국에서 돌아오는 집안인데, 집안 자체가 화려했던 그 시절을 잊지 못하고 씀씀이도 큰 편입니다...


추석경제
2차재난지원금 효과
안보임

돈을써도 쓰는효과
없도록 쓰느라 애썼다


경제와 방역 사이 헤매다.. 300조원 붓고도 경제불확실성 사상 최대
내년에도 코로나바이러스는 지속된다
코로나바이러스는 문재인과 같이 종식될지도 모른다


가능성 정도야 있다고 생각해. 근데 생각한다고 뭔가 바뀌는 건 아니지? 당장 나갈수도 없지? 안에 있다고 상황이 더 좋은것도 아니지? 그렇다면 믿지 않는 쪽이 경제적이라고 생각했는데~


다아는목적

바보작가소산

생존목숨을
위하여
돈경제를위하여
행복재미를
위하여
우리는
지금 여기에서
무엇을
위해
움직이는가?


경제적으로 독립하고싶다


세계의 정치와 경제를 암암리에 조종하는 패라디우스 사(社) 역시 도마의 힘에 의해 고대부터 축적된 부를 바탕으로 만들어진 기업이다


제 5 공화국 다시 보는데.. 전두환이라는 인간. 박정희라는 인간이 없었어도.. 경제 발전도 그대로였을테고..독재정권도 누가 또 만들었겠지. 다만 박정희는 몰라도 전두환은 그때 민주주의가 시작될 수 있었는데.. 그 다음 분기점은 3당 합당...


